# 50.007 Machine Learning, Spring 2025
# Design Project

Due 27 Apr 2025, 5:00pm

By: Aishwarya Iyer (1007141) and Khoo Zi Qi (1006984)

## Part1 (30points)

Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation): 

In [ ]:
# e(x|y) = Count(y → x)/ Count(y)

def emission_parameter():
    emission_parameters = count(y,x) / count(y)
    return emission_parameters